In [ ]:
#@title ## <font class="markdown-google-sans"><font color="red">❰</font> Open **`HTML`** webpage file</font>
import re
from pathlib import Path
from shutil import rmtree
from zipfile import ZipFile, ZIP_LZMA

colab = True
try:
	from google.colab import files
except ImportError:
	colab = False

# metadata
__author__ = "kubinka0505"
__version__ = "1.1"

# setup
BASE_DIR = Path("/content")
WORK_DIR = BASE_DIR / "workspace"

if WORK_DIR.exists():
	rmtree(WORK_DIR)

WORK_DIR.mkdir()
WORK_DIR.chdir() if hasattr(WORK_DIR, "chdir") else None

# helpers
def folder_size(path: Path) -> int:
	return sum(
		f.stat().st_size
		for f in path.rglob("*")
		if f.is_file() and not f.is_symlink()
	)


def human_size(bytes_: int) -> str:
	for unit in ("B", "KB", "MB", "GB", "TB"):
		if bytes_ < 1024:
			return f"{bytes_:.2f} {unit}"

		bytes_ /= 1024

	return f"{bytes_:.2f} PB"

def cubic_bezier(p0, p1, p2, p3, steps: int):
	points = []

	for i in range(1, steps + 1):
		t = i / steps

		x = (
			(1 - t)**3 * p0[0]
			+ 3 * (1 - t)**2 * t * p1[0]
			+ 3 * (1 - t) * t**2 * p2[0]
			+ t**3 * p3[0]
		)

		y = (
			(1 - t)**3 * p0[1]
			+ 3 * (1 - t)**2 * t * p1[1]
			+ 3 * (1 - t) * t**2 * p2[1]
			+ t**3 * p3[1]
		)
		points.append((x, y))

	return points

# files upload
if colab:
	files.upload()

	html_files = [
		f for f in Path(".").iterdir()
		if f.is_file() and f.suffix.lower() in (".html", ".htm")
	]
else:
	html_files = [os.sys.argv[1:]]

if not html_files:
	raise SystemExit("No HTML file uploaded")

html_file = html_files[0]

# parse canvas
content = html_file.read_text(encoding = "UTF-8")

start = content.find("ctx.moveTo(")
end = content.find("ctx.fill(")

if start == -1 or end == -1:
	raise SystemExit("Canvas drawing not found")

canvas_lines = content[start:end].splitlines()

# match ctx.moveTo(x, y) or ctx.lineTo(x, y)
move_re = re.compile(r"ctx\.moveTo\(\s*([\d.-]+)\s*,\s*([\d.-]+)\s*\)")
line_re = re.compile(r"ctx\.lineTo\(\s*([\d.-]+)\s*,\s*([\d.-]+)\s*\)")
bezier_re = re.compile(
	r"ctx\.bezierCurveTo\(\s*([\d.-]+)\s*,\s*([\d.-]+)\s*,"
	r"\s*([\d.-]+)\s*,\s*([\d.-]+)\s*,"
	r"\s*([\d.-]+)\s*,\s*([\d.-]+)\s*\)"
)

points = []
current = None
Steps = 25 #@param {type: "number"}

for line in canvas_lines:
	if m := move_re.search(line):
		current = (float(m[1]), float(m[2]))
		points.append(current)

	elif m := line_re.search(line):
		current = (float(m[1]), float(m[2]))
		points.append(current)

	elif m := bezier_re.search(line):
		p1 = (float(m[1]), float(m[2]))
		p2 = (float(m[3]), float(m[4]))
		p3 = (float(m[5]), float(m[6]))

		curve_pts = cubic_bezier(current, p1, p2, p3, Steps)
		points.extend(curve_pts)
		current = p3

if not points:
	raise SystemExit("No valid points found")

# output
output_dir = WORK_DIR / f"{html_file.stem}_Nodes"
output_dir.mkdir(exist_ok = True)

x_vals = [str(x) for x, _ in points]
y_vals = [str(y) for _, y in points]

x_vals.append(x_vals[0])
y_vals.append(y_vals[0])

(output_dir / "X.txt").write_text("\n".join(x_vals), encoding = "UTF-8")
(output_dir / "Y.txt").write_text("\n".join(y_vals), encoding = "UTF-8")

print(
	f"Successfully saved {len(points)} nodes "
	f"({human_size(folder_size(output_dir))})"
)


# download
if colab:
	zip_path = output_dir.with_suffix(".zip")

	with ZipFile(zip_path, "w", ZIP_LZMA) as z:
		for file in output_dir.iterdir():
			z.write(file, file.name)

	files.download(str(zip_path))